In [ ]:
import pandas as pd
import json

# Load the JSON file
with open('/content/DataEngineeringQ2.json', 'r') as file:
    data = json.load(file)

# Create an empty list to store the flattened data
flattened_data = []

# Iterate through each record in the JSON data
for record in data:
    patient_details = record.get('patientDetails', {})
    consultation_data = record.get('consultationData', {})
    medicines = consultation_data.get('medicines', [])

    # For each medicine in the consultationData, create a flattened record
    for medicine in medicines:
        flattened_record = {
            'appointmentId': record.get('appointmentId'),
            'patientId': patient_details.get('_id'),
            'firstName': patient_details.get('firstName'),
            'lastName': patient_details.get('lastName'),
            'emailId': patient_details.get('emailId'),
            'phoneNumber': record.get('phoneNumber'),
            'medicineId': medicine.get('medicineId'),
            'medicineName': medicine.get('medicineName'),
            'frequency': medicine.get('frequency'),
            'duration': medicine.get('duration'),
            'durationIn': medicine.get('durationIn'),
            'instruction': medicine.get('instruction'),
            'isActive': medicine.get('isActive')
        }
        flattened_data.append(flattened_record)

# Create a DataFrame from the flattened data
df = pd.DataFrame(flattened_data)

# Convert necessary columns to appropriate data types
df['duration'] = pd.to_numeric(df['duration'])
df['isActive'] = df['isActive'].astype(bool)

# Aggregation: Total duration of active medicines for each patient
active_medicines = df[df['isActive']]
agg_duration = active_medicines.groupby('patientId')['duration'].sum().reset_index()
agg_duration.columns = ['patientId', 'totalActiveDuration']

# Validate that all necessary fields are present
required_fields = ['appointmentId', 'patientId', 'firstName', 'medicineId', 'medicineName', 'frequency', 'duration', 'durationIn', 'instruction', 'isActive']
missing_fields = [field for field in required_fields if field not in df.columns]

if not missing_fields:
    print("All required fields are present.")
else:
    print(f"Missing fields: {missing_fields}")

# Validate the data types of the columns
print(df.dtypes)

# Check for any null values in critical columns
print(df[required_fields].isnull().sum())

# Insights: Most common medicine prescribed
common_medicine = df['medicineName'].value_counts().reset_index()
common_medicine.columns = ['medicineName', 'count']

# Insights: Average duration of medicines prescribed
avg_duration = df.groupby('medicineName')['duration'].mean().reset_index()
avg_duration.columns = ['medicineName', 'averageDuration']

# Print results
print("DataFrame Head:")
print(df.head())
print("\nTotal Active Duration per Patient:")
print(agg_duration.head())
print("\nMost Common Medicines:")
print(common_medicine.head())
print("\nAverage Duration of Medicines:")
print(avg_duration.head())


All required fields are present.
appointmentId    object
patientId        object
firstName        object
lastName         object
emailId          object
phoneNumber      object
medicineId       object
medicineName     object
frequency        object
duration          int64
durationIn       object
instruction      object
isActive           bool
dtype: object
appointmentId    0
patientId        0
firstName        0
medicineId       0
medicineName     0
frequency        0
duration         0
durationIn       0
instruction      0
isActive         0
dtype: int64
DataFrame Head:
  appointmentId  patientId firstName lastName emailId  phoneNumber medicineId  \
0     40d2-9c9f  T6hb630b3       Css                   96686896670     619404   
1     40d2-9c9f  T6hb630b3       Css                   96686896670     619804   
2      40dbtc9f  T6h33b300    Lokesh                    9496368916     619404   
3      40dbtc9f  T6h33b300    Lokesh                    9496368916     619804   
4      g3Wt5c9f  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Assuming data is the loaded JSON data

patient_details_data = []

# Iterate through each record in the JSON data to extract patient details
for record in data:
    patient_details = record.get('patientDetails', {})
    flattened_record = {
        'patientId': patient_details.get('_id'),
        'firstName': patient_details.get('firstName', ''),
        'lastName': patient_details.get('lastName', ''),
        'birthDate': patient_details.get('birthDate', '')
    }
    patient_details_data.append(flattened_record)


In [ ]:
def calculate_missing_percentage(df, column):
    total = len(df)
    missing = df[column].apply(lambda x: x == '' or pd.isnull(x)).sum()
    return round((missing / total) * 100, 2)

def get_missing_percentages(patient_details_data):
    # Create a DataFrame from the patient details data
    patient_df = pd.DataFrame(patient_details_data)

    # Calculate the percentage of missing values for firstName, lastName, and DOB
    missing_percentages = {
        'firstName': calculate_missing_percentage(patient_df, 'firstName'),
        'lastName': calculate_missing_percentage(patient_df, 'lastName'),
        'birthDate': calculate_missing_percentage(patient_df, 'birthDate')
    }

    return missing_percentages

# Assuming patient_details_data is already defined
missing_percentages = get_missing_percentages(patient_details_data)

# Print the missing percentages
print("Percentage of missing values:")
for column, percentage in missing_percentages.items():
    print(f"{column}: {percentage}%")


Percentage of missing values:
firstName: 0.0%
lastName: 70.97%
birthDate: 32.26%


In [ ]:
from collections import Counter

# Extract the gender data, ensuring we handle missing keys
gender_data = [entry['patientDetails'].get('gender', '') for entry in data]

# Calculate the mode of the gender column
mode_gender = Counter(gender_data).most_common(1)[0][0]

# Impute missing values with the mode
for entry in data:
    if 'gender' not in entry['patientDetails'] or entry['patientDetails']['gender'] == '':
        entry['patientDetails']['gender'] = mode_gender

# Calculate the percentage of female gender
total_entries = len(data)
female_count = sum(1 for entry in data if entry['patientDetails'].get('gender') == 'F')
female_percentage = (female_count / total_entries) * 100

# Round off to 2 decimal places
female_percentage = round(female_percentage, 2)

female_percentage



32.26

In [ ]:
from datetime import datetime

# Function to calculate age from birth date
def calculate_age(birth_date):
    if birth_date is None or birth_date == '':
        return None
    birth_date = datetime.strptime(birth_date, '%Y-%m-%dT%H:%M:%S.%fZ')
    today = datetime.today()
    return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))

# Function to categorize age into age groups
def categorize_age(age):
    if age is None:
        return 'Unknown'
    if age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teen'
    elif age <= 59:
        return 'Adult'
    else:
        return 'Senior'

# Add age and ageGroup to each entry
for entry in data:
    birth_date = entry['patientDetails'].get('birthDate', None)
    age = calculate_age(birth_date)
    entry['patientDetails']['age'] = age
    entry['patientDetails']['ageGroup'] = categorize_age(age)

# Calculate the count of adults
adult_count = sum(1 for entry in data if entry['patientDetails'].get('ageGroup') == 'Adult')

adult_count



21

In [ ]:
# Count the number of medicines prescribed per appointment
medicine_counts = df.groupby('appointmentId')['medicineId'].count().reset_index()
medicine_counts.columns = ['appointmentId', 'medicineCount']

# Calculate the average number of medicines prescribed
average_medicines = medicine_counts['medicineCount'].mean()

# Round the result to two decimal places
average_medicines_rounded = round(average_medicines, 2)

print(f"Average number of medicines prescribed: {average_medicines_rounded}")

Average number of medicines prescribed: 2.64


In [ ]:
# Count the occurrences of each medicine name
medicine_counts = df['medicineName'].value_counts()

# Check if there are at least 3 different medicines
if len(medicine_counts) >= 3:
    # Get the 3rd most frequent medicine
    third_most_frequent = medicine_counts.index[2]
    third_most_frequent_count = medicine_counts.iloc[2]

    print(f"3rd Most Frequent Medicine Name: {third_most_frequent}")
    print(f"Frequency: {third_most_frequent_count}")
else:
    print("There are fewer than 3 different medicines in the dataset.")

3rd Most Frequent Medicine Name: C
Frequency: 13


In [ ]:
# Count the total number of medicines
total_medicines = len(df)

# Count the number of active and inactive medicines
active_medicines_count = df['isActive'].sum()  # Since isActive is a boolean, summing gives count of True
inactive_medicines_count = total_medicines - active_medicines_count

# Calculate the percentage distribution
percent_active = (active_medicines_count / total_medicines) * 100
percent_inactive = (inactive_medicines_count / total_medicines) * 100

# Round the percentages to two decimal places
percent_active_rounded = round(percent_active, 2)
percent_inactive_rounded = round(percent_inactive, 2)

print(f"Percentage of Active Medicines: {percent_active_rounded}%")
print(f"Percentage of Inactive Medicines: {percent_inactive_rounded}%")

Percentage of Active Medicines: 69.7%
Percentage of Inactive Medicines: 30.3%


In [ ]:
import pandas as pd

# Assuming df already contains the loaded data and necessary columns

# Function to validate Indian phone numbers
def is_valid_indian_phone_number(phone_number):
    # Remove any spaces and hyphens for uniformity
    phone_number = phone_number.replace(' ', '').replace('-', '')

    # Check if the phone number starts with '+91 ' or '91'
    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]  # Remove '+91 '
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]  # Remove '91'
    else:
        return False

    # Check if the remaining part is a 10-digit number
    if len(phone_number) != 10 or not phone_number.isdigit():
        return False

    # Check if the number is within the valid range
    if 6000000000 <= int(phone_number) <= 9999999999:
        return True
    else:
        return False

# Apply function to create isValidMobile column
df['isValidMobile'] = df['phoneNumber'].apply(is_valid_indian_phone_number)

# Count number of valid phone numbers
valid_phone_count = df['isValidMobile'].sum()

# Output the number of valid phone numbers found in the dataset
print(valid_phone_count)

In [ ]:
import pandas as pd
from scipy.stats import pearsonr

# Assuming df already contains the loaded data and necessary columns
# Example placeholder for df creation
data = {
    'consultationData': [
        {'medicines': [{'medicineName': 'A'}, {'medicineName': 'B'}]},
        {'medicines': [{'medicineName': 'C'}]},
        {'medicines': []}
    ],
    'birthDate': ['1990-01-01', '1985-05-15', '1970-12-25']
}
df = pd.DataFrame(data)

# Function to extract number of prescribed medicines
def count_prescribed_medicines(consultation_data):
    if 'medicines' in consultation_data:
        return len(consultation_data['medicines'])
    else:
        return 0

# Apply function to count prescribed medicines for each row
df['prescribedMedicines'] = df['consultationData'].apply(count_prescribed_medicines)

# Function to calculate age from birthDate
def calculate_age(birth_date):
    # Implement logic to calculate age from birthDate
    # Example logic (assuming birth_date is in 'yyyy-mm-dd' format):
    # Replace this with actual age calculation based on current date
    return 2024 - int(birth_date[:4])

# Apply function to calculate age for each row
df['age'] = df['birthDate'].apply(calculate_age)

# Calculate Pearson correlation coefficient
pearson_corr, _ = pearsonr(df['prescribedMedicines'], df['age'])

# Round off to 2 decimal places
pearson_corr_rounded = round(pearson_corr, 2)

# Output the Pearson correlation coefficient
print(pearson_corr_rounded)